In [1]:
import requests
import spotipy
import pandas as pd
import math
import numpy  as np
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id ='f22713124fb04e97a26f46a51547744f',
                                                          client_secret='75c001d5dfc54f6cb6e44f7a54cdabc0'))

# Function defining

In [3]:
def get_multiple_playlists_from_genre(genres, limit = 5000):
    playlist_ids = list()
    for genre in genres:
        x = 0
        #cur_number_of_playlist = len(playlist_ids)
        results = sp.search(q=genre, type='playlist')
        for playlist in results['playlists']['items']:
            playlist_ids.append(playlist['id'])
            x+=1
        try:
            while results['playlists']['next'] and x < limit:
                #next page:
                #print(results['next'])
                results = sp.next(results['playlists'])
                for playlist in results['playlists']['items']:
                    playlist_ids.append(playlist['id'])
                    x+=1
        except:
            print('Counldnt go onto the next page for genre: ' + genre)
    return playlist_ids

In [4]:
def get_multiple_playlists_from_genre_incl_genres(genres,limit = 5000):
    playlist_ids = list()
    genre_list = list()
    for genre in genres:
        x = 0
        print(genre)
        #cur_number_of_playlist = len(playlist_ids)
        results = sp.search(q=genre, type='playlist')
        for playlist in results['playlists']['items']:
            playlist_ids.append(playlist['id'])
            genre_list.append(genre)
            x+=1
        try:
            while results['playlists']['next'] and x < limit:
                #next page:
                #print(results['next'])
                results = sp.next(results['playlists'])
                for playlist in results['playlists']['items']:
                    playlist_ids.append(playlist['id'])
                    genre_list.append(genre)
                    x+=1
        except:
            print('Counldnt go onto the next page for genre: ' + genre)
        print(x)
        print(str(len(playlist_ids)) + "und"+ str(len(genre_list)))
    return playlist_ids, genre_list

In [5]:
def get_playlist_tracks(playlist_id):
    results = sp.playlist(playlist_id)['tracks']
    tracks = results['items']
    while results['next']:
        #next page:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [6]:
def get_user_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results['items']
    while results['next']:
        #next page:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [8]:
def get_track_genre(album_id,artists_list):
    if album_id is not None:
        try:
            result = sp.album(album_id)['genres']
        except:
            print("Probably ReadTimeoutError for " + album_id)
            album_genre = []
#         print(album_id)
#         print(artists_list)
        if album_genre:
            result = album_genre
        elif artists_list:
            artist_genre = []
            for artist in artists_list:
                try:
                    artist_genre.append(sp.artist(artist['id'])['genres'])
                except AttributeError:
                    print("AttributeError for " + album_id)
                except TypeError:
                    print("TypeError for " + album_id)
                except:
                    print("Probably ReadTimeoutError for " + album_id)
            result = artist_genre
        else:
            print('No genre for either album or artist')
            result = []
        return result
    elif artists_list:
        artist_genre = []
        for artist in artists_list:
            try:
                artist_genre.append(sp.artist(artist['id'])['genres'])
            except AttributeError:
                print("AttributeError for " + album_id)
            except TypeError:
                print("TypeError for " + album_id)
            except:
                print("Probably ReadTimeoutError for " + album_id)
        result = artist_genre
    else:
        return []

In [206]:
def get_track_genre_opt(album_ids):
    #multiple ablum ids
    if album_ids is not None:
        try:
            album_genre = list()
            for loop in range(0,len(album_ids),20): 
                #print(loop)
                if loop < len(album_ids)-19:
                    albums = sp.albums(album_ids[loop:loop+20])['albums']
                    artist_ids = [albums[i]['artists'][0]['id'] for i in range(0,len(albums),1)]
                    artists = sp.artists(artist_ids)['artists']
                    #print(artists)
                    #print(len(albums))
                    for counter,element in enumerate(albums):
                        if element['genres']:
                            album_genre.append(element['genres'])
                        else: # get the genre from artists:
#                             print("no genre for album")
#                             print(artists[counter]['genres'])
                            if artists[counter]['genres']:
                                album_genre.append(artists[counter]['genres'])
                            else:
#                                 print("no genre for artist")
                                album_genre.append(np.nan)
                else:
                    loop_end = len(album_ids)
                    albums = sp.albums(album_ids[loop:loop_end])['albums']
                    artist_ids = [albums[i]['artists'][0]['id'] for i in range(0,len(albums),1)]
#                     print("Loop:" + str(loop) + "\n Loop_end: " + str(loop_end) + "\n len albums: " + str(len(album_ids)))
#                     print(albums[i]['artists'][0]['id'] for i in range(0,len(albums),1))
                    artists = sp.artists(artist_ids)['artists']
                    for counter,element in enumerate(albums):
                        if element['genres']:
                            album_genre.append(element['genres'])
                        else: # get the genre from artists:
#                             print("no genre for album")
                            if artists[counter]['genres']:
                                album_genre.append(artists[counter]['genres'])
                            else:
#                                 print("no genre for artist")
                                album_genre.append(np.nan)
            return album_genre
        except:
#             print("Probably ReadTimeoutError for in track_genre")
#             print(loop)
            return [np.nan]*100
    else:
#         print("cant be")
        return [np.nan]*100

In [10]:
def check_for_None(res,audio):
    if res is not None: 
        return res[audio]
    else:
        return np.nan

In [11]:
def check_for_None_in_list(res):
    x = 0
    while x < len(res):
        if list(res[x].keys())[0] is None:
            res.pop(x)
            x-=1
        x+=1
    return res  

In [12]:
def check_for_None_track(track_list):
    x = 0
    while x < len(track_list):
        if track_list[x]['track'] is None:
            track_list.pop(x)
            x-=1
        elif track_list[x]['track']['album'] is None:
            track_list.pop(x)
            x-=1
        elif track_list[x]['track']['artists'] is None:
            track_list.pop(x)
            x-=1
        elif track_list[x]['track']['album']['id'] is None:
            track_list.pop(x)
            x-=1
        x+=1
    return track_list

In [13]:
def get_track_features_from_user(username):
    df_tracks = pd.DataFrame()
    result = sp.user_playlists(username)
    playlists = result['items']
    while result['next']:
        #next page:
        result = sp.next(result)
        playlists.extend(result['items'])    
    for playlist in playlists:
        try:
            df_temp = get_track_features_from_user_playlist(username,playlist['id'])
            print(df_temp.shape)
        except TypeError:
            print('NoneType Object in audio features for Playlist_id: ' + playlist['id'])
        except AttributeError:
            print('AttributeError for NoneType Object in audio features for Playlist_id: ' + playlist['id'])
        except IndexError:
            print('IndexError in audio features for Playlist_id: ' + playlist['id'])
        df_tracks = pd.concat([df_tracks,df_temp], axis = 0)
    return df_tracks

In [14]:
def get_track_features_from_multiple_playlist(playlists):
    df_tracks = pd.DataFrame()
    for counter,playlist in enumerate(list(playlists['playlist_id'])):
        print(playlist + " " +str(counter) + " " + playlists.iloc[counter,1])
        try:
            df_temp = get_track_features_from_playlist(playlist)
            print(df_temp.shape)
            df_tracks = pd.concat([df_tracks,df_temp], axis = 0)
        except TypeError:
            print('NoneType Object in audio features for Playlist_id: ' + playlist)
        except AttributeError:
            print('AttributeError for NoneType Object in audio features for Playlist_id: ' + playlist)
        except IndexError:
            print('IndexError in audio features for Playlist_id: ' + playlist)
        except:
            print('Anything shit happened')
    return df_tracks

In [265]:
def get_track_features_from_multiple_playlist_opt(playlists):
    df_tracks = pd.DataFrame()
    for counter,playlist in enumerate(list(playlists['playlist_id'])):
        print(playlist + " " +str(counter) + " " + playlists.iloc[counter,1])
        try:
            df_temp = get_track_features_from_playlist_opt(playlist)
            print(df_temp.shape)
            df_tracks = pd.concat([df_tracks,df_temp], axis = 0)
        except TypeError:
            print('NoneType Object in audio features for Playlist_id: ' + playlist)
        except AttributeError:
            print('AttributeError for NoneType Object in audio features for Playlist_id: ' + playlist)
        except IndexError:
            print('IndexError in audio features for Playlist_id: ' + playlist)
        except:
            print('Anything shit happened')
    return df_tracks

In [264]:
def get_track_features_from_playlist_opt(playlist_id):
    #get all tracks from playlist:
    all_tracks = get_playlist_tracks(playlist_id)
    all_tracks = check_for_None_track(all_tracks)
    if all_tracks:
        #Check for None in Tracklist:
        all_tracks = check_for_None_track(all_tracks)
        #extract header for audio features:
        audio_keys = list(sp.audio_features(all_tracks[0]['track']['id'])[0].keys())
        #no need of url and other colums:
        del audio_keys[11:16]
        #create header for df:
        df_header = ['track_id','name','artist','genre']
        df_header.extend(audio_keys)
        #create empty df with header names:
        df_song = pd.DataFrame(columns = df_header)
        #create dictionary as <song_id> : <songname>, <artist_name>, <genre> of either track/album/artist>
        dic_id_song_artist_genre = [{track['track']['id'] : [
                                    track['track']['name'],
                                    track['track']['artists'][0]['name']]} 
                                    for track in all_tracks]
        dic_id_song_artist_genre = check_for_None_in_list(dic_id_song_artist_genre)
        #get all song ids:
        song_ids = [list(dic_id_song_artist_genre[i].keys())[0] for i in range(len(dic_id_song_artist_genre))]
        album_ids = [track['track']['album']['id'] for track in all_tracks]
        #print(len(album_ids))
        #loop through song_ids, for each song, get audio features and append to dataframe:
        for loop in range(0,len(song_ids),100): 
    #         print(loop)
    #         print(len(song_ids))
            if loop < len(song_ids)-99:
                result_audio = sp.audio_features(song_ids[loop:loop+100])
                result_genre = get_track_genre_opt(album_ids[loop:loop+100])
                #print(result_genre)
                dic_new_rows = {df_header[0] : [song_ids[i] for i in range(loop,loop+100,1)], 
                    df_header[1] : [dic_id_song_artist_genre[i][song_ids[i]][0] for i in range(loop,loop+100,1)],
                    df_header[2] : [dic_id_song_artist_genre[i][song_ids[i]][1] for i in range(loop,loop+100,1)],
                    df_header[3] : [result_genre[i] for i in range(0,len(result_genre),1)]}
                #print(len(result_audio))
                dic_new_rows.update({audio_keys[z] : [check_for_None(result_audio[x],audio_keys[z]) 
                                    for x in range(len(result_audio))] for z in range(len(audio_keys))})
                df_song = pd.concat([df_song,pd.DataFrame(dic_new_rows)], axis = 0)
            else:
                loop_end = len(song_ids)
                result_audio = sp.audio_features(song_ids[loop:loop_end])
                result_genre = get_track_genre_opt(album_ids[loop:loop_end])
                #print(result_genre)
    #             print(len(result))
    #             print(type(result_audio))
    #             print(len(audio_keys))
    #             #print(audio_keys)
    #             print([type(result[x]) for x in range(len(result))])
                #print(len(result_audio))
                dic_new_rows = {df_header[0] : [song_ids[i] for i in range(loop,loop_end,1)], 
                    df_header[1] : [dic_id_song_artist_genre[i][song_ids[i]][0] for i in range(loop,loop_end,1)],
                    df_header[2] : [dic_id_song_artist_genre[i][song_ids[i]][1] for i in range(loop,loop_end,1)],
                    df_header[3] : [result_genre[i] for i in range(0,len(result_genre),1)]}
    #             print([audio_keys[z] for z in range(len(audio_keys))])
    #             print([result[x][audio_keys[1]] for x in range(len(result)-1)])
                dic_new_rows.update({audio_keys[z] : [check_for_None(result_audio[x],audio_keys[z])
                                    for x in range(len(result_audio))] for z in range(len(audio_keys))})
                #[[print(result_audio[x][audio_keys[z]]) for x in range(len(result_audio))] for z in range(len(audio_keys))]
                df_song = pd.concat([df_song,pd.DataFrame(dic_new_rows)], axis = 0)
        return df_song
    else:
        return pd.DataFrame()

In [18]:
def get_track_features_from_playlist(playlist_id):
    #get all tracks from playlist:
    all_tracks = get_playlist_tracks(playlist_id)
    if all_tracks:
        #Check for None in Tracklist:
        all_tracks = check_for_None_track(all_tracks)
        #extract header for audio features:
        audio_keys = list(sp.audio_features(all_tracks[0]['track']['id'])[0].keys())
        #no need of url and other colums:
        del audio_keys[11:16]
        #create header for df:
        df_header = ['track_id','name','artist','genre']
        df_header.extend(audio_keys)
        #create empty df with header names:
        df_song = pd.DataFrame(columns = df_header)
        #create dictionary as <song_id> : <songname>, <artist_name>, <genre> of either track/album/artist>
        dic_id_song_artist_genre = [{track['track']['id'] : [
                                    track['track']['name'],
                                    track['track']['artists'][0]['name'],
                                    get_track_genre(track['track']['album']['id'],track['track']['artists'])]} 
                                    for track in all_tracks]
        dic_id_song_artist_genre = check_for_None_in_list(dic_id_song_artist_genre)
        #get all song ids:
        song_ids = [list(dic_id_song_artist_genre[i].keys())[0] for i in range(len(dic_id_song_artist_genre))]
        #loop through song_ids, for each song, get audio features and append to dataframe:
        for loop in range(0,len(song_ids),100): 
    #         print(loop)
    #         print(len(song_ids))
            if loop < len(song_ids)-99:
                result_audio = sp.audio_features(song_ids[loop:loop+100])
                dic_new_rows = {df_header[0] : [song_ids[i] for i in range(loop,loop+100,1)], 
                    df_header[1] : [dic_id_song_artist_genre[i][song_ids[i]][0] for i in range(loop,loop+100,1)],
                    df_header[2] : [dic_id_song_artist_genre[i][song_ids[i]][1] for i in range(loop,loop+100,1)],
                    df_header[3] : [dic_id_song_artist_genre[i][song_ids[i]][2][0] for i in range(loop,loop+100,1)]}
                #print(len(result_audio))
                dic_new_rows.update({audio_keys[z] : [check_for_None(result_audio[x],audio_keys[z]) 
                                    for x in range(len(result_audio))] for z in range(len(audio_keys))})
                df_song = pd.concat([df_song,pd.DataFrame(dic_new_rows)], axis = 0)
            else:
                loop_end = len(song_ids)
                result_audio = sp.audio_features(song_ids[loop:loop_end])
    #             print(len(result))
    #             print(type(result_audio))
    #             print(len(audio_keys))
    #             #print(audio_keys)
    #             print([type(result[x]) for x in range(len(result))])
                #print(len(result_audio))
                dic_new_rows = {df_header[0] : [song_ids[i] for i in range(loop,loop_end,1)], 
                    df_header[1] : [dic_id_song_artist_genre[i][song_ids[i]][0] for i in range(loop,loop_end,1)],
                    df_header[2] : [dic_id_song_artist_genre[i][song_ids[i]][1] for i in range(loop,loop_end,1)],
                    df_header[3] : [dic_id_song_artist_genre[i][song_ids[i]][2][0] for i in range(loop,loop_end,1)]}
    #             print([audio_keys[z] for z in range(len(audio_keys))])
    #             print([result[x][audio_keys[1]] for x in range(len(result)-1)])
                dic_new_rows.update({audio_keys[z] : [check_for_None(result_audio[x],audio_keys[z])
                                    for x in range(len(result_audio))] for z in range(len(audio_keys))})
                #[[print(result_audio[x][audio_keys[z]]) for x in range(len(result_audio))] for z in range(len(audio_keys))]
                df_song = pd.concat([df_song,pd.DataFrame(dic_new_rows)], axis = 0)
        return df_song
    else:
        return pd.DataFrame()

In [19]:
# def get_track_features_from_user_playlist(username,playlist_id):
#     #get all tracks from playlist:
#     all_tracks = get_user_playlist_tracks(username, playlist_id)
#     if all_tracks:
#         #Check for None in Tracklist:
#         all_tracks = check_for_None_track(all_tracks)
#         #extract header for audio features:
#         audio_keys = list(sp.audio_features(all_tracks[0]['track']['id'])[0].keys())
#         #no need of url and other colums:
#         del audio_keys[11:16]
#         #create header for df:
#         df_header = ['track_id','name','artist','genre']
#         df_header.extend(audio_keys)
#         #create empty df with header names:
#         df_song = pd.DataFrame(columns = df_header)
#         #create dictionary as <song_id> : <songname>, <artist_name>, <genre> of either track/album/artist>
#         dic_id_song_artist_genre = [{track['track']['id'] : [
#                                     track['track']['name'],
#                                     track['track']['artists'][0]['name'],
#                                     get_track_genre(track['track']['album']['id'],track['track']['artists'])]} 
#                                     for track in all_tracks]
#         dic_id_song_artist_genre = check_for_None_in_list(dic_id_song_artist_genre)
#         #get all song ids:
#         song_ids = [list(dic_id_song_artist_genre[i].keys())[0] for i in range(len(dic_id_song_artist_genre))]
#         #loop through song_ids, for each song, get audio features and append to dataframe:
#         for loop in range(0,len(song_ids),100): 
#     #         print(loop)
#     #         print(len(song_ids))
#             if loop < len(song_ids)-99:
#                 result_audio = sp.audio_features(song_ids[loop:loop+100])
#                 dic_new_rows = {df_header[0] : [song_ids[i] for i in range(loop,loop+100,1)], 
#                     df_header[1] : [dic_id_song_artist_genre[i][song_ids[i]][0] for i in range(loop,loop+100,1)],
#                     df_header[2] : [dic_id_song_artist_genre[i][song_ids[i]][1] for i in range(loop,loop+100,1)],
#                     df_header[3] : [dic_id_song_artist_genre[i][song_ids[i]][2][0] for i in range(loop,loop+100,1)]}
#                 #print(len(result_audio))
#                 dic_new_rows.update({audio_keys[z] : [check_for_None(result_audio[x],audio_keys[z]) 
#                                     for x in range(len(result_audio))] for z in range(len(audio_keys))})
#                 df_song = pd.concat([df_song,pd.DataFrame(dic_new_rows)], axis = 0)
#             else:
#                 loop_end = len(song_ids)
#                 result_audio = sp.audio_features(song_ids[loop:loop_end])
#     #             print(len(result))
#     #             print(type(result_audio))
#     #             print(len(audio_keys))
#     #             #print(audio_keys)
#     #             print([type(result[x]) for x in range(len(result))])
#                 #print(len(result_audio))
#                 dic_new_rows = {df_header[0] : [song_ids[i] for i in range(loop,loop_end,1)], 
#                     df_header[1] : [dic_id_song_artist_genre[i][song_ids[i]][0] for i in range(loop,loop_end,1)],
#                     df_header[2] : [dic_id_song_artist_genre[i][song_ids[i]][1] for i in range(loop,loop_end,1)],
#                     df_header[3] : [dic_id_song_artist_genre[i][song_ids[i]][2][0] for i in range(loop,loop_end,1)]}
#     #             print([audio_keys[z] for z in range(len(audio_keys))])
#     #             print([result[x][audio_keys[1]] for x in range(len(result)-1)])
#                 dic_new_rows.update({audio_keys[z] : [check_for_None(result_audio[x],audio_keys[z])
#                                     for x in range(len(result_audio))] for z in range(len(audio_keys))})
#                 #[[print(result_audio[x][audio_keys[z]]) for x in range(len(result_audio))] for z in range(len(audio_keys))]
#                 df_song = pd.concat([df_song,pd.DataFrame(dic_new_rows)], axis = 0)
#         return df_song
#     else:
#         return pd.DataFrame()

## Defining Music Categories:

In [20]:
music_genre = ["Blues","Country", "Dark Music","Disco", "Electro","Folk", "Funk", "Hip-Hop","Jazz",
               "Latin", "Metal", "Pop","Punk","Reggae","Rock", "Rock ’n’ Roll", "Ska", "Traditional"]

In [21]:
len(music_genre)

18

## Getting lots of Songs from Spotify User

In [18]:
# X = get_track_features_from_user('spotify')

In [19]:
# X.head()

In [20]:
# X.shape

In [306]:
# df_fullsonglist = X.copy()

In [307]:
# df_fullsonglist = df_fullsonglist.drop_duplicates(subset = 'track_id')

In [255]:
# df_fullsonglist.shape

In [556]:
# df_fullsonglist.to_csv('full_songlist_without_duplicates_from_spotify_user.csv')
# df_fullsonglist.to_pickle('full_songlist_without_duplicates_from_spotify_user.pickle')

# Playlist seeking:

In [259]:
# playlist_ids_for_multiple_genres, genres = get_multiple_playlists_from_genre_incl_genres(music_genre,limit=200)
# df_playlists_genre = pd.DataFrame({"playlist_id" : playlist_ids_for_multiple_genres,"genre" : genres})
# df_playlists_genre = df_playlists_genre.drop_duplicates(subset = 'playlist_id', ignore_index=True)
# df_playlists_genre.to_pickle('playlist_ids_limit200_multiple_genres_incl_genre.pickle')
df_playlists_genre = pd.read_pickle('playlist_ids_limit200_multiple_genres_incl_genre.pickle')

In [23]:
# df_playlists_genre.tail(50)

In [34]:
# playlist_ids_for_multiple_genres = get_multiple_playlists_from_genre(music_genre,limit=200)
# df_playlists = pd.DataFrame(playlist_ids_for_multiple_genres, columns= ["playlist_id"])
# df_playlists = df_playlists.drop_duplicates(subset = 'playlist_id', ignore_index=True)
# df_playlists.to_pickle('playlist_ids_limit200_multiple_genres.pickle')

In [274]:
df_playlists_genre.head()

,playlist_id,genre
0,37i9dQZF1DXd9rSDyQguIk,Blues
1,37i9dQZF1DWSKpvyAAcaNZ,Blues
2,37i9dQZF1DX2iUghHXGIjj,Blues
3,37i9dQZF1DXcFk5r8uS3l2,Blues
4,4xKFUXoma8EepzgxKguJUi,Blues


In [24]:
# df_playlists.shape

## Getting lots of songs from multiple Playlists:

In [273]:
X_full = get_track_features_from_multiple_playlist_opt(df_playlists_genre.loc[3000:3050])

3gN4rp6CtRNYeIwW23s4HR 0 Rock ’n’ Roll
NoneType Object in audio features for Playlist_id: 3gN4rp6CtRNYeIwW23s4HR
60yALU7Z5dv9EQbCs8m9rs 1 Rock ’n’ Roll
NoneType Object in audio features for Playlist_id: 60yALU7Z5dv9EQbCs8m9rs
1Jmunx1gLB1bJkBghL7yFY 2 Rock ’n’ Roll
NoneType Object in audio features for Playlist_id: 1Jmunx1gLB1bJkBghL7yFY
37i9dQZF1DX4KurNW74MNE 3 Rock ’n’ Roll
NoneType Object in audio features for Playlist_id: 37i9dQZF1DX4KurNW74MNE
1ZGka1ThUkKhcMz8sj8EAT 4 Rock ’n’ Roll
NoneType Object in audio features for Playlist_id: 1ZGka1ThUkKhcMz8sj8EAT
1vGo9fjaiUrSU0HTvrDEhl 5 Rock ’n’ Roll
NoneType Object in audio features for Playlist_id: 1vGo9fjaiUrSU0HTvrDEhl
53tuAdmHXSjjDfrUeTD7aY 6 Rock ’n’ Roll
NoneType Object in audio features for Playlist_id: 53tuAdmHXSjjDfrUeTD7aY
3wIS0tvZVZNVF3oxJwUZJM 7 Rock ’n’ Roll
NoneType Object in audio features for Playlist_id: 3wIS0tvZVZNVF3oxJwUZJM
5fdI2O2EOgOgBs3YZdwaJ5 8 Rock ’n’ Roll
NoneType Object in audio features for Playlist_id: 5fdI2O

In [249]:
df_fullsonglist = X_full.copy()

In [268]:
df_fullsonglist.head(3)

,track_id,name,artist,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,4c0Mgs559lG4sPiEC3XcxY,To Know You Is To Love You,B.B. King,"[blues, blues rock, classic rock, electric blu...",0.872,0.384,0,-16.204,1,0.0388,0.418,0.317000,0.0687,0.938,102.597,515000,4
1,01UTGeKaSANZCXxkAu6vPY,Born Under A Bad Sign,Albert King,"[blues, blues rock, classic rock, electric blu...",0.699,0.633,11,-6.249,1,0.0261,0.436,0.000093,0.0291,0.907,91.398,167040,4
2,2Qj1FilGsXITZbFVmVeoLy,I'm Tore Down,Freddie King,"[blues, blues rock, classic rock, electric blu...",0.640,0.496,2,-11.124,1,0.0413,0.616,0.000046,0.1830,0.954,116.632,253333,4


In [252]:
print(df_fullsonglist.shape[0])
df_fullsonglist = df_fullsonglist.drop_duplicates(subset = 'track_id')
print(df_fullsonglist.shape[0])

519184
331063


In [253]:
df_fullsonglist.reset_index(drop=True)

,track_id,name,artist,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,4c0Mgs559lG4sPiEC3XcxY,To Know You Is To Love You,B.B. King,"[blues, blues rock, classic rock, electric blu...",0.872,0.384,0,-16.204,1,0.0388,0.4180,0.317000,0.0687,0.938,102.597,515000,4
1,01UTGeKaSANZCXxkAu6vPY,Born Under A Bad Sign,Albert King,"[blues, blues rock, classic rock, electric blu...",0.699,0.633,11,-6.249,1,0.0261,0.4360,0.000093,0.0291,0.907,91.398,167040,4
2,2Qj1FilGsXITZbFVmVeoLy,I'm Tore Down,Freddie King,"[blues, blues rock, classic rock, electric blu...",0.640,0.496,2,-11.124,1,0.0413,0.6160,0.000046,0.1830,0.954,116.632,253333,4
3,5MTsZG10E0hTvAkdioyPnw,Feels Like Rain,Buddy Guy,"[blues, blues rock, chicago blues, classic roc...",0.572,0.348,0,-13.454,1,0.0293,0.4380,0.000731,0.2350,0.340,73.303,275267,4
4,6f3OYG5R630tjwFIHIE7j2,Scratch My Back,The Fabulous Thunderbirds,"[blues, blues rock, classic rock, country rock...",0.775,0.499,4,-7.181,1,0.0415,0.4960,0.021900,0.1140,0.862,121.056,232707,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331058,0ExpuH39K9SnZtCL6viN1t,Ehi Ena Feggari Apopse,Pashalis Terzis,"[greek pop, laiko]",0.747,0.594,5,-5.160,0,0.0329,0.5980,0.000000,0.3200,0.693,122.057,218693,5
331059,3t36me1JNGMpDDzfOuCBVV,Den Tha Xanagapiso,Stelios Kazantzidis,"[classic greek pop, greek pop, laiko, rebetiko]",0.542,0.539,7,-8.486,0,0.0395,0.6270,0.000006,0.1690,0.590,139.653,192000,4
331060,5rZAgfcv0vpOZ0eQR1jdvF,Ipirhan Orki,Giannis Ploutarhos,"[greek pop, laiko]",0.754,0.702,9,-6.048,0,0.0444,0.1110,0.000000,0.2920,0.606,124.916,184933,4
331061,44wbOvHi9VDQd9NBaEXWQK,Diskola Feggaria,Giorgos Mazonakis,"[greek pop, laiko]",0.710,0.643,9,-7.645,0,0.0253,0.0579,0.000002,0.1030,0.595,133.977,281443,4


In [272]:
df_fullsonglist[df_fullsonglist['track_id']=="4X7tJyJkP3Vh1Yilt3V778"]

,track_id,name,artist,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
24,4X7tJyJkP3Vh1Yilt3V778,Rip It Up,Rock Explosion & The Shaking Hearts,[indorock],0.46,0.74,4,-7.27,1,0.0378,0.229,0.0,0.228,0.93,199.842,156373,4


In [254]:
df_fullsonglist.to_pickle('full_songlist_all_genres.pickle')